# 批量预测

1. 需要指定要批量预测的样本。

   1. 自己手动指定。
   2. 通过`mydir`参数循环找到指定目录下的所有图片数据。
   
2. 指定Onekey训练好的目录路径，对应到`config_path参数`，精确到`viz`目录。
> 例如：path2yours/20220802/inception_v3_bb/viz

In [10]:
import os
from onekey_algo import get_param_in_cwd
from glob import glob
import pandas as pd

POSSIBLE_IMG = ['.jpg', '.jpeg', '.png', '.bmp']

#data_root = get_param_in_cwd('data_root')
data_root = r'F:\DATA_primary'
#config_path = os.path.join(data_root, 'models', 'resnet18', 'viz')
config_path = r'D:\bianyun_onekey\models\resnet18\viz'
train = pd.read_csv(os.path.join(data_root, 'split_info', 'train-RND-0.txt'), sep='\t', header=None, names=['fname', 'label'])
test = pd.read_csv(os.path.join(data_root, 'split_info', 'val-RND-0.txt'), sep='\t', header=None, names=['fname', 'label'])
#already = set([os.path.basename(s) for s in set(train['fname']) | set(test['fname'])])
# 打印 already 集合
#print("Already set:", len(already))



#all_samples = glob(os.path.join(data_root, 'patches', '*/*.jpg'))
all_samples = glob(os.path.join(data_root, 'patches', '*/*.jpg'))

print("All samples:", all_samples[:10])  # 打印前 10 个文件路径

#all_samples = glob(r'E:\bian_data\*/*.jpg')

#samples = [s for s in all_samples if os.path.basename(s) not in already]
#print("Filtered samples:", samples[:10])  # 打印前 10 个过滤后的文件路径

samples = all_samples
print(len(samples))





All samples: ['F:\\DATA_primary\\patches\\1238060-D\\1238060-D_43008_43008_10240_10240_10752_10752.jpg', 'F:\\DATA_primary\\patches\\1238060-D\\1238060-D_43008_43008_10240_10752_10752_11264.jpg', 'F:\\DATA_primary\\patches\\1238060-D\\1238060-D_43008_43008_10240_11264_10752_11776.jpg', 'F:\\DATA_primary\\patches\\1238060-D\\1238060-D_43008_43008_10240_11776_10752_12288.jpg', 'F:\\DATA_primary\\patches\\1238060-D\\1238060-D_43008_43008_10240_12288_10752_12800.jpg', 'F:\\DATA_primary\\patches\\1238060-D\\1238060-D_43008_43008_10240_12800_10752_13312.jpg', 'F:\\DATA_primary\\patches\\1238060-D\\1238060-D_43008_43008_10240_13312_10752_13824.jpg', 'F:\\DATA_primary\\patches\\1238060-D\\1238060-D_43008_43008_10240_13824_10752_14336.jpg', 'F:\\DATA_primary\\patches\\1238060-D\\1238060-D_43008_43008_10240_14336_10752_14848.jpg', 'F:\\DATA_primary\\patches\\1238060-D\\1238060-D_43008_43008_10240_14848_10752_15360.jpg']
1054325


In [11]:
import json
from onekey_algo.classification.eval_classification import init as clf_init
from onekey_algo.classification.eval_classification import inference_dataloader as clf_inference

model, transformer, labels, device = clf_init(config_path=config_path)
results = clf_inference(samples, model, transformer, labels, device, num_workers=20, batch_size=64)

r = pd.DataFrame(results, columns=['fname', 'prob', 'pred'])
r['gt'] = 1
r['prob'] = r['prob'].map(lambda x: json.loads(x)['1']).round(decimals=2)
r.to_csv(os.path.join(data_root, 'models', 'resnet18', 'viz', 'predictions_DATA_primary.csv'), index=False)
r[['group']] = r[['fname']].applymap(lambda x: x.split('_')[0])


[2024-06-28 16:37:27 - eval_classification.py:  65]	INFO	正在预测中，已完成：64000, 完成率：6.0702%，移动平均速度是：3.5218 msec/img
[2024-06-28 16:38:51 - eval_classification.py:  65]	INFO	正在预测中，已完成：128000, 完成率：12.1405%，移动平均速度是：2.4185 msec/img
[2024-06-28 16:40:15 - eval_classification.py:  65]	INFO	正在预测中，已完成：192000, 完成率：18.2107%，移动平均速度是：2.0462 msec/img
[2024-06-28 16:41:40 - eval_classification.py:  65]	INFO	正在预测中，已完成：256000, 完成率：24.2809%，移动平均速度是：1.8683 msec/img
[2024-06-28 16:43:03 - eval_classification.py:  65]	INFO	正在预测中，已完成：320000, 完成率：30.3512%，移动平均速度是：1.7530 msec/img
[2024-06-28 16:44:26 - eval_classification.py:  65]	INFO	正在预测中，已完成：384000, 完成率：36.4214%，移动平均速度是：1.6776 msec/img
[2024-06-28 16:45:51 - eval_classification.py:  65]	INFO	正在预测中，已完成：448000, 完成率：42.4916%，移动平均速度是：1.6277 msec/img
[2024-06-28 16:47:17 - eval_classification.py:  65]	INFO	正在预测中，已完成：512000, 完成率：48.5619%，移动平均速度是：1.5919 msec/img
[2024-06-28 16:48:41 - eval_classification.py:  65]	INFO	正在预测中，已完成：576000, 完成率：54.6321%，移动平均速度是：1.5616 mse

# 移动特征

In [6]:
r.to_csv(os.path.join(data_root, 'models', 'resnet18', 'viz', 'predictions_TCGA.csv'), index=False)

In [7]:
import shutil
import os

#features_dir = os.path.join(data_root, 'models/SimpleViT/viz/features')\
features_dir = os.path.join(r'D:\bianyun_onekey\models\SimpleViT\viz\features')
for f in os.listdir(features_dir):
    fn = os.path.splitext(f)[0]
    shutil.move(os.path.join(features_dir, f), os.path.join(features_dir, f"{fn.split('-')[0]}.csv"))

In [8]:
label_data = pd.read_csv(os.path.join(data_root, 'split_info/label-RND-0.csv'), dtype={'ID': str})

for _, row in label_data.iterrows():
    group = row['group']
    save2 = os.path.join(features_dir, group)
    os.makedirs(save2, exist_ok=True)
    shutil.move(os.path.join(features_dir, f"{row['ID']}.csv"), save2)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\bianyun_onekey\\models\\SimpleViT\\viz\\features\\TCGA-HZ-7926.csv'